# Proyecto Dataset Diabético

## 📊 Resumen del Dataset

- **Dimensiones**: 101,766 registros × 50 columnas
- **Variable objetivo**: `readmitted` (NO, <30 días, >30 días)
- **Tipo de problema**: Clasificación multiclase (puede convertirse a binaria)

---


In [32]:
import pandas as pd
import numpy as np
import warnings
from sklearn.impute import SimpleImputer
warnings.filterwarnings('ignore')

## Limpieza y Preprocesamiento de Datos

In [33]:
df = pd.read_csv("data/diabetic_data.csv")
    
print(f"\n✓ Dataset cargado exitosamente")
print(f"  Dimensiones originales: {df.shape[0]} filas × {df.shape[1]} columnas")


✓ Dataset cargado exitosamente
  Dimensiones originales: 101766 filas × 50 columnas


### Inspección Inicial del Dataset

In [34]:
# Información general del dataset
print("=" * 80)
print("INFORMACIÓN GENERAL DEL DATASET")
print("=" * 80)
df.info()

INFORMACIÓN GENERAL DEL DATASET
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            1

In [35]:
# Primeras filas del dataset
print("\n" + "=" * 80)
print("PRIMERAS 5 FILAS DEL DATASET")
print("=" * 80)
display(df.head())


PRIMERAS 5 FILAS DEL DATASET


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [36]:
# Estadísticas descriptivas
print("\n" + "=" * 80)
print("ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 80)
display(df.describe())


ESTADÍSTICAS DESCRIPTIVAS


,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,1.017660e+05,1.017660e+05,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,1.652016e+08,5.433040e+07,2.024006,3.715642,5.754437,4.395987,43.095641,1.339730,16.021844,0.369357,0.197836,0.635566,7.422607
std,1.026403e+08,3.869636e+07,1.445403,5.280166,4.064081,2.985108,19.674362,1.705807,8.127566,1.267265,0.930472,1.262863,1.933600
min,1.252200e+04,1.350000e+02,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,8.496119e+07,2.341322e+07,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,1.523890e+08,4.550514e+07,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,2.302709e+08,8.754595e+07,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,4.438672e+08,1.895026e+08,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000


In [37]:
# Análisis de valores faltantes
print("\n" + "=" * 80)
print("ANÁLISIS DE VALORES FALTANTES")
print("=" * 80)

# Valores '?' que se consideran como faltantes
missing_counts = (df == '?').sum()
missing_counts = missing_counts[missing_counts > 0].sort_values(ascending=False)

if len(missing_counts) > 0:
    print("\nColumnas con '?' (valores faltantes):")
    for col, count in missing_counts.items():
        pct = (count / len(df)) * 100
        print(f"  {col:30s}: {count:6d} ({pct:5.2f}%)")
else:
    print("No hay valores '?' en el dataset")

# Valores NaN tradicionales
nan_counts = df.isnull().sum()
nan_counts = nan_counts[nan_counts > 0].sort_values(ascending=False)

if len(nan_counts) > 0:
    print("\nColumnas con NaN:")
    for col, count in nan_counts.items():
        pct = (count / len(df)) * 100
        print(f"  {col:30s}: {count:6d} ({pct:5.2f}%)")
else:
    print("\nNo hay valores NaN en el dataset")


ANÁLISIS DE VALORES FALTANTES

Columnas con '?' (valores faltantes):
  weight                        :  98569 (96.86%)
  medical_specialty             :  49949 (49.08%)
  payer_code                    :  40256 (39.56%)
  race                          :   2273 ( 2.23%)
  diag_3                        :   1423 ( 1.40%)
  diag_2                        :    358 ( 0.35%)
  diag_1                        :     21 ( 0.02%)

Columnas con NaN:
  max_glu_serum                 :  96420 (94.75%)
  A1Cresult                     :  84748 (83.28%)


In [38]:
# Análisis de duplicados
print("\n" + "=" * 80)
print("ANÁLISIS DE DUPLICADOS")
print("=" * 80)

# Duplicados por encounter_id (cada encuentro debe ser único)
duplicate_encounters = df['encounter_id'].duplicated().sum()
print(f"Registros duplicados por encounter_id: {duplicate_encounters}")

# Duplicados por patient_nbr (un paciente puede tener múltiples encuentros)
unique_patients = df['patient_nbr'].nunique()
total_encounters = len(df)
print(f"Pacientes únicos: {unique_patients}")
print(f"Total de encuentros: {total_encounters}")
print(f"Promedio de encuentros por paciente: {total_encounters / unique_patients:.2f}")


ANÁLISIS DE DUPLICADOS
Registros duplicados por encounter_id: 0
Pacientes únicos: 71518
Total de encuentros: 101766
Promedio de encuentros por paciente: 1.42


### Demostración de Operaciones Vectorizadas con NumPy

In [39]:
import time

# Convertir columna numérica a array de NumPy
time_in_hospital = df['time_in_hospital'].values

print("=" * 80)
print("COMPARACIÓN: OPERACIONES VECTORIZADAS vs LOOPS TRADICIONALES")
print("=" * 80)

# Operación 1: Calcular suma usando loop tradicional
start_time = time.time()
total_loop = 0
for value in time_in_hospital:
    total_loop += value
time_loop = time.time() - start_time

# Operación 1: Calcular suma usando NumPy (vectorizado)
start_time = time.time()
total_numpy = np.sum(time_in_hospital)
time_numpy = time.time() - start_time

print(f"\n1. SUMA DE VALORES:")
print(f"   Loop tradicional: {total_loop:.2f} - Tiempo: {time_loop*1000:.4f} ms")
print(f"   NumPy vectorizado: {total_numpy:.2f} - Tiempo: {time_numpy*1000:.4f} ms")
if time_numpy > 0:
    print(f"   Speedup: {time_loop/time_numpy:.1f}x más rápido")
else:
    print(f"   Speedup: NumPy es extremadamente rápido (< 0.01ms)")

# Operación 2: Calcular media usando loop
start_time = time.time()
mean_loop = sum(time_in_hospital) / len(time_in_hospital)
time_loop = time.time() - start_time

# Operación 2: Calcular media usando NumPy
start_time = time.time()
mean_numpy = np.mean(time_in_hospital)
time_numpy = time.time() - start_time

print(f"\n2. MEDIA:")
print(f"   Loop tradicional: {mean_loop:.2f} - Tiempo: {time_loop*1000:.4f} ms")
print(f"   NumPy vectorizado: {mean_numpy:.2f} - Tiempo: {time_numpy*1000:.4f} ms")
if time_numpy > 0:
    print(f"   Speedup: {time_loop/time_numpy:.1f}x más rápido")
else:
    print(f"   Speedup: NumPy es extremadamente rápido (< 0.01ms)")

# Operación 3: Normalización Min-Max usando loop
start_time = time.time()
min_val = min(time_in_hospital)
max_val = max(time_in_hospital)
normalized_loop = [(x - min_val) / (max_val - min_val) for x in time_in_hospital]
time_loop = time.time() - start_time

# Operación 3: Normalización Min-Max usando NumPy
start_time = time.time()
normalized_numpy = (time_in_hospital - np.min(time_in_hospital)) / (np.max(time_in_hospital) - np.min(time_in_hospital))
time_numpy = time.time() - start_time

print(f"\n3. NORMALIZACIÓN MIN-MAX:")
print(f"   Loop tradicional: Tiempo: {time_loop*1000:.4f} ms")
print(f"   NumPy vectorizado: Tiempo: {time_numpy*1000:.4f} ms")
if time_numpy > 0:
    print(f"   Speedup: {time_loop/time_numpy:.1f}x más rápido")
else:
    print(f"   Speedup: NumPy es extremadamente rápido (< 0.01ms)")

print(f"\n✓ Las operaciones vectorizadas de NumPy son significativamente más rápidas")

COMPARACIÓN: OPERACIONES VECTORIZADAS vs LOOPS TRADICIONALES

1. SUMA DE VALORES:
   Loop tradicional: 447362.00 - Tiempo: 26.9496 ms
   NumPy vectorizado: 447362.00 - Tiempo: 0.3104 ms
   Speedup: 86.8x más rápido

2. MEDIA:
   Loop tradicional: 4.40 - Tiempo: 10.1466 ms
   NumPy vectorizado: 4.40 - Tiempo: 1.0841 ms
   Speedup: 9.4x más rápido

3. NORMALIZACIÓN MIN-MAX:
   Loop tradicional: Tiempo: 46.7839 ms
   NumPy vectorizado: Tiempo: 2.6407 ms
   Speedup: 17.7x más rápido

✓ Las operaciones vectorizadas de NumPy son significativamente más rápidas


### Eliminación de columnas con mayoría de nulos

Dado que la columna ``weight`` tiene un 97% de nulos, consideramos que la mejor estrategia es eliminarla.

In [40]:
# Drop columna 'weight'
df = df.drop(columns=['weight'])
print(f"  Dimensiones después de eliminar 'weight': {df.shape[0]} filas × {df.shape[1]} columnas")

  Dimensiones después de eliminar 'weight': 101766 filas × 49 columnas


### Imputación de Nulos Menores

**Imputación de ``race`` con la moda**

In [41]:
# Paso 1: Convertir '?' a NaN solo en la columna 'race'
df['race'] = df['race'].replace('?', np.nan)


In [42]:
# Paso 2: Backup y verificación
print("Valores nulos originales en 'race':", df['race'].isna().sum())
df['race_backup'] = df['race'].copy()

Valores nulos originales en 'race': 2273


In [43]:
# Paso 3: Imputación
race_imputer = SimpleImputer(strategy='most_frequent')
df['race'] = race_imputer.fit_transform(df[['race']]).ravel()

In [44]:
# Paso 4: Identificar valores imputados
imputed_mask = df['race_backup'].isna() & df['race'].notna()
imputed_values = df.loc[imputed_mask, 'race']

print(f"\nSe imputaron {len(imputed_values)} valores:")
print(imputed_values.value_counts())


Se imputaron 2273 valores:
race
Caucasian    2273
Name: count, dtype: int64


### Sustitución de nulos en ``payer_code`` y ``medical_specialty`` por ``unknown``

In [45]:
# Sustituir '?' por 'Unknown' en payer_code y medical_specialty
df['payer_code'] = df['payer_code'].replace('?', 'Unknown')
df['medical_specialty'] = df['medical_specialty'].replace('?', 'Unknown')

print(f"✓ Valores '?' reemplazados por 'Unknown' en payer_code y medical_specialty")
print(f"  payer_code: {(df['payer_code'] == 'Unknown').sum()} valores 'Unknown'")
print(f"  medical_specialty: {(df['medical_specialty'] == 'Unknown').sum()} valores 'Unknown'")

✓ Valores '?' reemplazados por 'Unknown' en payer_code y medical_specialty
  payer_code: 40256 valores 'Unknown'
  medical_specialty: 49949 valores 'Unknown'


### Transformación de Variable Objetivo

In [46]:
# Opción 1: Readmitido vs No readmitido ¿?
df['readmitted_binary'] = (df['readmitted'] != 'NO').astype(int)

# Opción 2: Readmitido en <30 días vs resto  ¿?
df['early_readmission'] = (df['readmitted'] == '<30').astype(int)

### Encoding de Variables Categóricas

In [47]:
# Identificar columnas categóricas y numéricas
print("=" * 80)
print("PREPARACIÓN PARA ENCODING")
print("=" * 80)

# Columnas a excluir del encoding (IDs y columnas ya procesadas)
exclude_cols = ['encounter_id', 'patient_nbr', 'readmitted', 'readmitted_binary', 'early_readmission']

# Identificar columnas categóricas (object type)
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
categorical_cols = [col for col in categorical_cols if col not in exclude_cols]

print(f"\nColumnas categóricas identificadas ({len(categorical_cols)}):")
for col in categorical_cols:
    unique_count = df[col].nunique()
    print(f"  {col:30s}: {unique_count:3d} valores únicos")

# Separar entre nominales (muchos valores) y binarias/ordinales
high_cardinality_cols = [col for col in categorical_cols if df[col].nunique() > 10]
low_cardinality_cols = [col for col in categorical_cols if df[col].nunique() <= 10]

print(f"\nColumnas de alta cardinalidad (>10 valores): {len(high_cardinality_cols)}")
print(f"Columnas de baja cardinalidad (≤10 valores): {len(low_cardinality_cols)}")

PREPARACIÓN PARA ENCODING

Columnas categóricas identificadas (36):
  race                          :   5 valores únicos
  gender                        :   3 valores únicos
  age                           :  10 valores únicos
  payer_code                    :  18 valores únicos
  medical_specialty             :  73 valores únicos
  diag_1                        : 717 valores únicos
  diag_2                        : 749 valores únicos
  diag_3                        : 790 valores únicos
  max_glu_serum                 :   3 valores únicos
  A1Cresult                     :   3 valores únicos
  metformin                     :   4 valores únicos
  repaglinide                   :   4 valores únicos
  nateglinide                   :   4 valores únicos
  chlorpropamide                :   4 valores únicos
  glimepiride                   :   4 valores únicos
  acetohexamide                 :   2 valores únicos
  glipizide                     :   4 valores únicos
  glyburide                    

In [48]:
# Aplicar Label Encoding a columnas de baja cardinalidad
from sklearn.preprocessing import LabelEncoder

print("\n" + "=" * 80)
print("LABEL ENCODING (para columnas de baja cardinalidad)")
print("=" * 80)

label_encoders = {}
df_encoded = df.copy()

for col in low_cardinality_cols:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
    label_encoders[col] = le
    print(f"✓ {col}: {len(le.classes_)} clases codificadas")

print(f"\n✓ Total de columnas con Label Encoding: {len(low_cardinality_cols)}")


LABEL ENCODING (para columnas de baja cardinalidad)
✓ race: 5 clases codificadas
✓ gender: 3 clases codificadas
✓ age: 10 clases codificadas
✓ max_glu_serum: 4 clases codificadas
✓ A1Cresult: 4 clases codificadas
✓ metformin: 4 clases codificadas
✓ repaglinide: 4 clases codificadas
✓ nateglinide: 4 clases codificadas
✓ chlorpropamide: 4 clases codificadas
✓ glimepiride: 4 clases codificadas
✓ acetohexamide: 2 clases codificadas
✓ glipizide: 4 clases codificadas
✓ glyburide: 4 clases codificadas
✓ tolbutamide: 2 clases codificadas
✓ pioglitazone: 4 clases codificadas
✓ rosiglitazone: 4 clases codificadas
✓ acarbose: 4 clases codificadas
✓ miglitol: 4 clases codificadas
✓ troglitazone: 2 clases codificadas
✓ tolazamide: 3 clases codificadas
✓ examide: 1 clases codificadas
✓ citoglipton: 1 clases codificadas
✓ insulin: 4 clases codificadas
✓ glyburide-metformin: 4 clases codificadas
✓ glipizide-metformin: 2 clases codificadas
✓ glimepiride-pioglitazone: 2 clases codificadas
✓ metformin-r

In [49]:
# Aplicar One-Hot Encoding a columnas de alta cardinalidad
print("\n" + "=" * 80)
print("ONE-HOT ENCODING (para columnas de alta cardinalidad)")
print("=" * 80)

# Para evitar explosión dimensional, limitaremos las columnas de alta cardinalidad
# o usaremos frequency encoding como alternativa

print(f"\nColumnas de alta cardinalidad que se procesarán:")
for col in high_cardinality_cols:
    print(f"  {col}: {df_encoded[col].nunique()} valores únicos")

# Decisión: Para payer_code (18 valores) aplicaremos One-Hot
# Para las demás columnas con demasiados valores (diagnosis codes),
# usaremos Label Encoding ya que One-Hot crearía demasiadas columnas

# Columnas para One-Hot (cardinalidad moderada)
onehot_cols = ['payer_code']  # 18 valores únicos es manejable
onehot_cols = [col for col in onehot_cols if col in high_cardinality_cols]

if onehot_cols:
    df_encoded = pd.get_dummies(df_encoded, columns=onehot_cols, prefix=onehot_cols, drop_first=True)
    print(f"\n✓ One-Hot Encoding aplicado a: {onehot_cols}")
    print(f"  Nuevas columnas creadas: {len([c for c in df_encoded.columns if any(pc in c for pc in onehot_cols)])}")
else:
    print("\n⚠️  No hay columnas seleccionadas para One-Hot Encoding")

# Para las demás columnas de alta cardinalidad, aplicar Label Encoding
remaining_high_card = [col for col in high_cardinality_cols if col not in onehot_cols]
for col in remaining_high_card:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
    label_encoders[col] = le
    print(f"✓ Label Encoding aplicado a {col}")

print(f"\n✓ Encoding completo. Nueva dimensión: {df_encoded.shape}")


ONE-HOT ENCODING (para columnas de alta cardinalidad)

Columnas de alta cardinalidad que se procesarán:
  payer_code: 18 valores únicos
  medical_specialty: 73 valores únicos
  diag_1: 717 valores únicos
  diag_2: 749 valores únicos
  diag_3: 790 valores únicos

✓ One-Hot Encoding aplicado a: ['payer_code']
  Nuevas columnas creadas: 17
✓ Label Encoding aplicado a medical_specialty
✓ Label Encoding aplicado a diag_1
✓ Label Encoding aplicado a diag_2
✓ Label Encoding aplicado a diag_3

✓ Encoding completo. Nueva dimensión: (101766, 68)


### Feature Engineering

In [50]:
# Crear nuevas características combinadas
print("=" * 80)
print("FEATURE ENGINEERING")
print("=" * 80)

# Feature 1: Total de visitas (outpatient + emergency + inpatient)
df_encoded['total_visits'] = (df_encoded['number_outpatient'] + 
                               df_encoded['number_emergency'] + 
                               df_encoded['number_inpatient'])
print(f"✓ Feature creada: total_visits")
print(f"  Rango: [{df_encoded['total_visits'].min()}, {df_encoded['total_visits'].max()}]")
print(f"  Media: {df_encoded['total_visits'].mean():.2f}")

# Feature 2: Cambios en medicación (change + diabetesMed)
# Primero, convertir a numérico si no lo son
if df_encoded['change'].dtype == 'object':
    df_encoded['change'] = df_encoded['change'].map({'No': 0, 'Ch': 1})
if df_encoded['diabetesMed'].dtype == 'object':
    df_encoded['diabetesMed'] = df_encoded['diabetesMed'].map({'No': 0, 'Yes': 1})

df_encoded['medication_changes'] = df_encoded['change'] + df_encoded['diabetesMed']
print(f"\n✓ Feature creada: medication_changes")
print(f"  Rango: [{df_encoded['medication_changes'].min()}, {df_encoded['medication_changes'].max()}]")
print(f"  Distribución:")
print(df_encoded['medication_changes'].value_counts().sort_index())

# Feature 3: Ratio de procedimientos por día hospitalizado
df_encoded['procedures_per_day'] = df_encoded['num_procedures'] / (df_encoded['time_in_hospital'] + 1)  # +1 para evitar división por 0
print(f"\n✓ Feature creada: procedures_per_day")
print(f"  Media: {df_encoded['procedures_per_day'].mean():.2f}")

print(f"\n✓ Feature Engineering completado. Total de features: {df_encoded.shape[1]}")

FEATURE ENGINEERING
✓ Feature creada: total_visits
  Rango: [0, 80]
  Media: 1.20

✓ Feature creada: medication_changes
  Rango: [1, 2]
  Distribución:
medication_changes
1    70414
2    31352
Name: count, dtype: int64

✓ Feature creada: procedures_per_day
  Media: 0.30

✓ Feature Engineering completado. Total de features: 71


### Standard Scaler para variables numéricas

In [51]:
from sklearn.preprocessing import StandardScaler

print("=" * 80)
print("NORMALIZACIÓN DE VARIABLES NUMÉRICAS")
print("=" * 80)

scaler = StandardScaler()

# Identificar columnas numéricas (excluyendo las variables objetivo y IDs)
numerical_cols = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 
                  'num_medications', 'number_outpatient', 'number_emergency', 
                  'number_inpatient', 'number_diagnoses', 'total_visits', 
                  'medication_changes', 'procedures_per_day']

# Verificar que las columnas existen
numerical_cols = [col for col in numerical_cols if col in df_encoded.columns]

print(f"\nColumnas numéricas a normalizar ({len(numerical_cols)}):")
for col in numerical_cols:
    print(f"  - {col}")

# Aplicar StandardScaler
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])

print(f"\n✓ Normalización completada")
print(f"  Media después de escalar (debe ser ~0): {df_encoded[numerical_cols].mean().mean():.6f}")
print(f"  Desviación estándar (debe ser ~1): {df_encoded[numerical_cols].std().mean():.6f}")

NORMALIZACIÓN DE VARIABLES NUMÉRICAS

Columnas numéricas a normalizar (11):
  - time_in_hospital
  - num_lab_procedures
  - num_procedures
  - num_medications
  - number_outpatient
  - number_emergency
  - number_inpatient
  - number_diagnoses
  - total_visits
  - medication_changes
  - procedures_per_day

✓ Normalización completada
  Media después de escalar (debe ser ~0): 0.000000
  Desviación estándar (debe ser ~1): 1.000005


In [52]:
# Decodificación inteligente de códigos ICD-9 (diag_1, diag_2, diag_3)
# Crear flags binarios por grupo clínico y un `comorbidity_score` agregando las condiciones encontradas.

# Asegurar que `df_encoded` existe (si no, crear a partir de `df` cargado previamente)
if 'df_encoded' not in globals():
    if 'df' in globals():
        df_encoded = df.copy()
        print("df_encoded no existía: creado a partir de df")
    else:
        raise NameError("Neither df_encoded nor df found in the kernel. Ejecuta las celdas previas que cargan el dataset antes de correr esta celda.")


def _prefix_int(code):
    """Return the integer prefix of an ICD-9 code or None for non-numeric codes.
    Examples: '250.01' -> 250, '428' -> 428, 'V45' -> None
    """
    if pd.isna(code):
        return None
    s = str(code).strip()
    if s == '' or s == '?':
        return None
    # take part before dot
    pref = s.split('.')[0]
    # if it starts with non-digit (V, E), ignore for our groups
    if not pref[0].isdigit():
        return None
    try:
        return int(pref)
    except Exception:
        return None


def is_diabetes(code):
    p = _prefix_int(code)
    return p == 250


def is_circulatory(code):
    p = _prefix_int(code)
    if p is None:
        return False
    return (390 <= p <= 459) or (p == 785)


def is_respiratory(code):
    p = _prefix_int(code)
    if p is None:
        return False
    return (460 <= p <= 519) or (p == 786)

# Aplicar a las tres columnas de diagnóstico
for i in [1, 2, 3]:
    col = f"diag_{i}"
    # defensivo: si la columna no existe, crear con NaNs
    if col not in df_encoded.columns:
        df_encoded[col] = np.nan
    df_encoded[f"{col}_is_diabetes"] = df_encoded[col].apply(lambda x: 1 if is_diabetes(x) else 0)
    df_encoded[f"{col}_is_circulatory"] = df_encoded[col].apply(lambda x: 1 if is_circulatory(x) else 0)
    df_encoded[f"{col}_is_respiratory"] = df_encoded[col].apply(lambda x: 1 if is_respiratory(x) else 0)

# Flags a nivel paciente
# diabetes_primary: diag_1 es diabetes
if 'diag_1' in df_encoded.columns:
    df_encoded['diabetes_primary'] = df_encoded['diag_1'].apply(lambda x: 1 if is_diabetes(x) else 0)
else:
    df_encoded['diabetes_primary'] = 0

# diabetes_secondary: no es primaria pero aparece en diag_2 o diag_3
diag2_has = df_encoded['diag_2'].apply(lambda x: True if is_diabetes(x) else False) if 'diag_2' in df_encoded.columns else False
diag3_has = df_encoded['diag_3'].apply(lambda x: True if is_diabetes(x) else False) if 'diag_3' in df_encoded.columns else False

df_encoded['diabetes_secondary'] = (((df_encoded['diabetes_primary'] == 0) & (diag2_has | diag3_has))).astype(int)

# Any flags
df_encoded['diabetes_any'] = df_encoded[[f'diag_{i}_is_diabetes' for i in [1,2,3]]].max(axis=1)
df_encoded['circulatory_any'] = df_encoded[[f'diag_{i}_is_circulatory' for i in [1,2,3]]].max(axis=1)
df_encoded['respiratory_any'] = df_encoded[[f'diag_{i}_is_respiratory' for i in [1,2,3]]].max(axis=1)

# Comorbidity score: count how many of these major disease groups are present (0-3)
df_encoded['comorbidity_score'] = (df_encoded['diabetes_any'].astype(int) +
                                   df_encoded['circulatory_any'].astype(int) +
                                   df_encoded['respiratory_any'].astype(int))

# Informes rápidos
print("=" * 80)
print("ICD-9 DECODING Y COMORBIDITY FEATURES")
print("=" * 80)
print(f"Total registros: {len(df_encoded)}")
print("\nSuma de banderas principales (número de registros que tienen la condición):")
print(df_encoded[['diabetes_any','diabetes_primary','diabetes_secondary','circulatory_any','respiratory_any','comorbidity_score']].sum())
print("\nComorbidity score - valores y porcentajes:")
print(df_encoded['comorbidity_score'].value_counts().sort_index())
print((df_encoded['comorbidity_score'].value_counts(normalize=True)*100).sort_index())

# Mostrar algunas filas de ejemplo
print("\n" + "-"*80)
print("VISTA PREVIA: columnas relevantes para comorbilidades")
display(df_encoded[[ 'diag_1','diag_2','diag_3', 'diabetes_primary','diabetes_secondary', 'diabetes_any','circulatory_any','respiratory_any','comorbidity_score']].head(10))

# Re-guardar el dataset limpio incluyendo las nuevas features
output_path = "data/diabetes_clean.csv"
df_encoded.to_csv(output_path, index=False)
print(f"\n✓ Dataset limpio (actualizado) guardado en: {output_path}")


ICD-9 DECODING Y COMORBIDITY FEATURES
Total registros: 101766

Suma de banderas principales (número de registros que tienen la condición):
diabetes_any            146
diabetes_primary         30
diabetes_secondary      116
circulatory_any       20551
respiratory_any       17007
comorbidity_score     37704
dtype: int64

Comorbidity score - valores y porcentajes:
comorbidity_score
0    66998
1    31835
2     2930
3        3
Name: count, dtype: int64
comorbidity_score
0    65.835348
1    31.282550
2     2.879154
3     0.002948
Name: proportion, dtype: float64

--------------------------------------------------------------------------------
VISTA PREVIA: columnas relevantes para comorbilidades


,diag_1,diag_2,diag_3,diabetes_primary,diabetes_secondary,diabetes_any,circulatory_any,respiratory_any,comorbidity_score
0,124,650,670,0,0,0,0,0,0
1,143,79,121,0,0,0,0,0,0
2,454,78,767,0,0,0,1,0,1
3,554,97,248,0,0,0,0,0,0
4,54,24,86,0,0,0,0,0,0
5,263,246,86,0,0,0,0,0,0
6,263,246,771,0,0,0,0,0,0
7,276,314,86,0,0,0,0,0,0
8,252,260,229,0,0,0,0,0,0
9,282,46,317,0,0,0,0,0,0



✓ Dataset limpio (actualizado) guardado en: data/diabetes_clean.csv


In [53]:
# --- CODIGO DE EXTENSIÓN: DIGESTIVE & NEOPLASMS ---

# 1. Definimos las nuevas funciones de chequeo basadas en los rangos ICD-9 estándar
def is_digestive(code):
    """Códigos 520-579 (Enf. Sistema Digestivo) y 787 (Síntomas sistema digestivo)"""
    p = _prefix_int(code)
    if p is None: return False
    return (520 <= p <= 579) or (p == 787)

def is_neoplasms(code):
    """Códigos 140-239 (Tumores / Cáncer)"""
    p = _prefix_int(code)
    if p is None: return False
    return (140 <= p <= 239)

def is_injury(code):
    """Códigos 800-999 (Lesiones y envenenamientos) - Opcional pero recomendado"""
    p = _prefix_int(code)
    if p is None: return False
    return (800 <= p <= 999)

# 2. Aplicamos a las columnas diag_1, diag_2, diag_3
# (Asumimos que df_encoded ya tiene las anteriores procesadas)

groups_to_add = [
    ('digestive', is_digestive),
    ('neoplasms', is_neoplasms),
    ('injury', is_injury) # Puedes quitar esta línea si no quieres incluir lesiones
]

for col_name, func in groups_to_add:
    # Flag por cada columna de diagnóstico (diag_1, diag_2, diag_3)
    for i in [1, 2, 3]:
        df_encoded[f'diag_{i}_is_{col_name}'] = df_encoded[f'diag_{i}'].apply(lambda x: 1 if func(x) else 0)
    
    # Flag global "ANY" (¿Tiene esta condición en alguno de los 3 diagnósticos?)
    cols_to_check = [f'diag_{i}_is_{col_name}' for i in [1, 2, 3]]
    df_encoded[f'{col_name}_any'] = df_encoded[cols_to_check].max(axis=1)

# 3. ACTUALIZACIÓN DEL COMORBIDITY SCORE
# Ahora sumamos las 3 anteriores + las nuevas. El score máximo sube.
# (Nota: Asegúrate de tener calculadas diabetes_any, circulatory_any, respiratory_any del paso anterior)

df_encoded['comorbidity_score_v2'] = (
    df_encoded['diabetes_any'].astype(int) +
    df_encoded['circulatory_any'].astype(int) +
    df_encoded['respiratory_any'].astype(int) +
    df_encoded['digestive_any'].astype(int) +
    df_encoded['neoplasms_any'].astype(int) +
    df_encoded['injury_any'].astype(int)
)

# --- REPORTING DE RESULTADOS ---
print("=" * 60)
print("NUEVAS COMORBILIDADES AGREGADAS")
print("=" * 60)
print("\nConteo de pacientes con las nuevas condiciones (ANY):")
print(df_encoded[['digestive_any', 'neoplasms_any', 'injury_any']].sum())

print("\nComparación de Scores:")
print("Score Original (0-3) vs Nuevo Score V2 (0-6)")
print("-" * 40)
print(df_encoded['comorbidity_score_v2'].value_counts().sort_index())

# Verificamos un paciente que tenga Neoplasms para ver si funcionó
print("\nEjemplo de pacientes con Neoplasms (Cáncer):")
example_neoplasms = df_encoded[df_encoded['neoplasms_any'] == 1][['diag_1', 'diag_2', 'neoplasms_any', 'comorbidity_score_v2']].head(5)
display(example_neoplasms)

NUEVAS COMORBILIDADES AGREGADAS

Conteo de pacientes con las nuevas condiciones (ANY):
digestive_any    16330
neoplasms_any    24240
injury_any           0
dtype: int64

Comparación de Scores:
Score Original (0-3) vs Nuevo Score V2 (0-6)
----------------------------------------
comorbidity_score_v2
0    40129
1    45795
2    15047
3      795
Name: count, dtype: int64

Ejemplo de pacientes con Neoplasms (Cáncer):


,diag_1,diag_2,neoplasms_any,comorbidity_score_v2
1,143,79,1,1
8,252,260,1,1
11,26,145,1,1
22,259,239,1,2
25,540,239,1,2


In [54]:
# Eliminar columnas innecesarias antes de guardar
print("=" * 80)
print("PREPARACIÓN FINAL Y GUARDADO")
print("=" * 80)

# Eliminar la columna original 'readmitted' y columnas de backup
cols_to_drop = ['readmitted']
if 'race_backup' in df_encoded.columns:
    cols_to_drop.append('race_backup')

df_encoded = df_encoded.drop(cols_to_drop, axis=1, errors='ignore')

print(f"\n✓ Columnas eliminadas: {cols_to_drop}")
print(f"  Dimensiones finales: {df_encoded.shape[0]} filas × {df_encoded.shape[1]} columnas")

# Guardar el dataset limpio
output_path = "data/diabetes_clean.csv"
df_encoded.to_csv(output_path, index=False)
print(f"\n✓ Dataset limpio guardado en: {output_path}")

# Mostrar vista previa
print("\n" + "=" * 80)
print("VISTA PREVIA DEL DATASET LIMPIO")
print("=" * 80)
display(df_encoded.head())

# Información final
print("\n" + "=" * 80)
print("RESUMEN FINAL")
print("=" * 80)
print(f"Total de registros: {len(df_encoded)}")
print(f"Total de features: {df_encoded.shape[1]}")
print(f"\nVariables objetivo disponibles:")
print(f"  - readmitted_binary: Clasificación binaria (readmitido vs no readmitido)")
print(f"  - early_readmission: Readmisión temprana <30 días (más crítica)")
print(f"\nDistribución de early_readmission:")
print(df_encoded['early_readmission'].value_counts())
print(f"\nTipos de datos:")
print(df_encoded.dtypes.value_counts())

PREPARACIÓN FINAL Y GUARDADO

✓ Columnas eliminadas: ['readmitted', 'race_backup']
  Dimensiones finales: 101766 filas × 97 columnas

✓ Dataset limpio guardado en: data/diabetes_clean.csv

VISTA PREVIA DEL DATASET LIMPIO


,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,...,digestive_any,diag_1_is_neoplasms,diag_2_is_neoplasms,diag_3_is_neoplasms,neoplasms_any,diag_1_is_injury,diag_2_is_injury,diag_3_is_injury,injury_any,comorbidity_score_v2
0,2278392,8222157,2,0,0,6,25,1,-1.137649,37,...,0,0,0,0,0,0,0,0,0,0
1,149190,55629189,2,0,1,1,1,7,-0.467653,71,...,0,1,0,0,1,0,0,0,0,1
2,64410,86047875,0,0,2,1,1,7,-0.802651,71,...,0,0,0,0,0,0,0,0,0,1
3,500364,82442376,2,1,3,1,1,7,-0.802651,71,...,1,0,0,0,0,0,0,0,0,1
4,16680,42519267,2,1,4,1,1,7,-1.137649,71,...,0,0,0,0,0,0,0,0,0,0



RESUMEN FINAL
Total de registros: 101766
Total de features: 97

Variables objetivo disponibles:
  - readmitted_binary: Clasificación binaria (readmitido vs no readmitido)
  - early_readmission: Readmisión temprana <30 días (más crítica)

Distribución de early_readmission:
early_readmission
0    90409
1    11357
Name: count, dtype: int64

Tipos de datos:
int64      69
bool       17
float64    11
Name: count, dtype: int64


### TODO: Análisis del Desbalance de Clases


### Balanceo con SMOTE o class_weight = 'balanced' (VA JUSTO DESPUÉS DE HACER LA DIVISIÓN DE ENTRENAMIENTO )
**Nota:** El balanceo de clases se manejará durante el entrenamiento de modelos usando `class_weight='balanced'` en los algoritmos o aplicando SMOTE si es necesario.